# Convert new .model and .json files to huggingface tokenizer

In [4]:
import json
from transformers import AutoTokenizer
from train_tokenizer import read_training_info
from tokenize_simple import load_pretokenizer


/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

def convert_tokenizer_to_huggingface(new_tokenizer_info_path: str, original_tokenizer: str):
    # must get original tokenizer from huggingface
    tokenizer = AutoTokenizer.from_pretrained(original_tokenizer)
    # load new_tokenizer_info_path
    new_tokenizer_info = read_training_info(new_tokenizer_info_path)
    # get merges and new_tokens
    merges = new_tokenizer_info["merges"]
    new_tokens = new_tokenizer_info["new_tokens"]
    # add new tokens to tokenizer
    tokenizer.add_tokens(new_tokens)
    return tokenizer

info_path = "/cmlscratch/astein0/efficient_tokenization_for_inference/tokenizers/Llama-3.2-tokenizer-genqa-math-empty-start/new_mergeable_ranks_128266.json"
original_tokenizer = "meta-llama/Llama-3.2-1B"

# tok = convert_tokenizer_to_huggingface(info_path, original_tokenizer)

/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_tokenizer_to_huggingface_correct(new_tokenizer_info_path: str, original_tokenizer: str):
    # must get original tokenizer from huggingface
    tokenizer = AutoTokenizer.from_pretrained(original_tokenizer)
    tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
    
    # old_vocab = tokenizer_json["model"]["vocab"]
    # starting_index = len(tokenizer.get_vocab())
    old_merges = tokenizer_json["model"]["merges"]

    # Extract vocab (token: index)
    old_vocab = tokenizer.get_vocab()
    
    # load new_tokenizer_info_path
    new_tokenizer_info = read_training_info(new_tokenizer_info_path)
    # get merges and new_tokens
    new_merges = new_tokenizer_info["merges"]
    new_tokens = new_tokenizer_info["new_tokens"]

    # Update vocab (append at the next available ID)
    new_vocab = {**old_vocab}  # Copy the old vocab
    starting_index = max(old_vocab.values()) + 1

    for i, token in enumerate(new_tokens):
        new_vocab[token] = starting_index + i

    new_vocab_sorted = dict(sorted(new_vocab.items(), key=lambda item: item[1]))
    # new_vocab_sorted = new_vocab

    joined_merges = [x for x in old_merges]
    joined_merges.extend(new_merges)

    added_tokens = tokenizer.get_added_vocab()
    add_tok_ids = [tok_id for tok, tok_id in added_tokens.items()]

    new_vocab_sorted_no_added = {tok: tok_id for  tok, tok_id in new_vocab_sorted.items() if tok_id not in add_tok_ids}

    new_tokenizer_info = {**tokenizer_json}
    new_tokenizer_info["model"]["vocab"] = new_vocab_sorted_no_added
    new_tokenizer_info["model"]["merges"] = joined_merges

    # add new tokens to tokenizer
    return new_tokenizer_info

info_path = "/cmlscratch/astein0/efficient_tokenization_for_inference/tokenizers/Llama-3.2-tokenizer-genqa-math-empty-start/new_mergeable_ranks_128266.json"
original_tokenizer_str = "meta-llama/Llama-3.2-1B"

new_tokenizer_json = convert_tokenizer_to_huggingface_correct(info_path, original_tokenizer_str)



tokenizer_object None


In [11]:
import os

# 2️⃣ Define new save path
new_tokenizer_path = "tokenizers/new_custom_tokenizer"
os.makedirs(new_tokenizer_path, exist_ok=True)

original_tokenizer = AutoTokenizer.from_pretrained(original_tokenizer_str)

# 3️⃣ Save the tokenizer to a directory
original_tokenizer.save_pretrained(new_tokenizer_path)

tokenizer_json_path = os.path.join(new_tokenizer_path, "tokenizer.json")
with open(tokenizer_json_path, "w") as f:
    json.dump(new_tokenizer_json, f, indent=2)



NameError: name 'original_tokenizer_str' is not defined

In [ ]:
# 5️⃣ Reload tokenizer to verify it works
new_tokenizer = AutoTokenizer.from_pretrained(new_tokenizer_path)

# 6️⃣ Check vocab size to confirm new tokens were added
print(f"✅ New tokenizer loaded! Vocab size: {len(new_tokenizer.get_vocab())}")


In [17]:
original_tokenizer = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(original_tokenizer)
tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
    
old_vocab = tokenizer_json["model"]["vocab"]
starting_index = len(tokenizer)


tokenizer_object None


In [7]:
pre_tok = load_pretokenizer("empty")



Sequence(pretokenizers=[ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=False)])

In [19]:
def pretokenizer_to_config(pre_tok):
    # Convert the pretokenizer string representation to a config dict
    str_rep = str(pre_tok)
    # Remove the outer Sequence() wrapper
    inner = str_rep[len("Sequence(pretokenizers=["):-2]
    
    # Parse ByteLevel config
    # Format is: ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=False)
    byte_level_config = {}
    byte_level_params = inner[len("ByteLevel("):-1].split(", ")
    for param in byte_level_params:
        key, value = param.split("=")
        byte_level_config[key] = value.lower() == "true"  # Convert string to boolean
    
    config = {
        "type": "Sequence",
        "pretokenizers": [{
            "type": "ByteLevel",
            **byte_level_config
        }]
    }
    return config

# Use it like this:
config = pretokenizer_to_config(pre_tok)
config

{'type': 'Sequence',
 'pretokenizers': [{'type': 'ByteLevel',
   'add_prefix_space': False,
   'trim_offsets': True,
   'use_regex': False}]}

In [ ]:
{"pre_tokenizer": {
    "type": "Sequence",
    "pretokenizers": [
      {
        "type": "Split",
        "pattern": {
          "Regex": "(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"
        },
        "behavior": "Isolated",
        "invert": false
      },
      {
        "type": "ByteLevel",
        "add_prefix_space": false,
        "trim_offsets": true,
        "use_regex": false
      }
    ]
}

In [5]:
def pretokenizer_to_config(pre_tok):
    """Convert any pretokenizer to a config dictionary by parsing its string representation."""
    str_rep = str(pre_tok)
    
    def parse_params(param_str):
        """Parse parameters from string representation into a dictionary."""
        params = {}
        # Handle empty parameters
        if not param_str:
            return params
            
        for param in param_str.split(", "):
            key, value = param.split("=")
            # Try to convert value to appropriate type
            if value.lower() in ['true', 'false']:
                params[key] = value.lower() == 'true'
            elif value.isdigit():
                params[key] = int(value)
            elif value.replace('.', '').isdigit():
                params[key] = float(value)
            else:
                # Remove quotes if present
                params[key] = value.strip("'\"")
        return params
    
    def parse_pretokenizer(tok_str):
        """Recursively parse a pretokenizer string into a config dictionary."""
        # Find the type and parameters
        tok_type = tok_str[:tok_str.find("(")]
        param_str = tok_str[tok_str.find("(")+1:tok_str.rfind(")")]
        
        # Handle nested pretokenizers (like in Sequence)
        if tok_type == "Sequence":
            # Extract the list of pretokenizers
            pretok_list = param_str[len("pretokenizers=["):-1]
            # Split on "), " but keep the closing parenthesis
            nested_toks = []
            current = ""
            paren_count = 0
            for char in pretok_list:
                if char == "(":
                    paren_count += 1
                elif char == ")":
                    paren_count -= 1
                current += char
                if paren_count == 0 and char == ")":
                    nested_toks.append(current)
                    current = ""
                elif paren_count == 0 and char == ",":
                    current = ""
                
            return {
                "type": tok_type,
                "pretokenizers": [parse_pretokenizer(tok.strip()) for tok in nested_toks if tok.strip()]
            }
        else:
            # Regular pretokenizer
            return {
                "type": tok_type,
                **parse_params(param_str)
            }
    
    return parse_pretokenizer(str_rep)

# Example usage:
from tokenizers import pre_tokenizers

# Test with different pretokenizer configurations
pretok_configs = [
    pre_tokenizers.ByteLevel(add_prefix_space=False, trim_offsets=True),
    pre_tokenizers.Whitespace(),
    pre_tokenizers.Sequence([
        pre_tokenizers.ByteLevel(add_prefix_space=False),
        pre_tokenizers.Whitespace()
    ]),
    pre_tokenizers.Metaspace(replacement="▁"),
    pre_tokenizers.Digits(individual_digits=True),
    load_pretokenizer("empty"),
    load_pretokenizer("llama3"),
]

# Test each configuration
for pre_tok in pretok_configs:
    config = pretokenizer_to_config(pre_tok)
    print(f"\nPretokenizer: {pre_tok}")
    print(f"Config: {config}")


Pretokenizer: ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=True)
Config: {'type': 'ByteLevel', 'add_prefix_space': False, 'trim_offsets': True, 'use_regex': True}

Pretokenizer: Whitespace()
Config: {'type': 'Whitespace'}

Pretokenizer: Sequence(pretokenizers=[ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=True), Whitespace()])
Config: {'type': 'Sequence', 'pretokenizers': [{'type': 'ByteLevel', 'add_prefix_space': False, 'trim_offsets': True, 'use_regex': True}, {'type': 'Whitespace'}]}

Pretokenizer: Metaspace(replacement="▁", prepend_scheme=always, split=True)
Config: {'type': 'Metaspace', 'replacement': '▁', 'prepend_scheme': 'always', 'split': True}

Pretokenizer: Digits(individual_digits=True)
Config: {'type': 'Digits', 'individual_digits': True}

Pretokenizer: Sequence(pretokenizers=[ByteLevel(add_prefix_space=False, trim_offsets=True, use_regex=False)])
Config: {'type': 'Sequence', 'pretokenizers': [{'type': 'ByteLevel', 'add_prefix_space': Fa

 # DUPLICATING the dataset

In [1]:
import tokenize_simple
from transformers import AutoTokenizer
import psutil
import datasets

batch_size = 500

try:
    threads = min(psutil.cpu_count(logical=False), len(psutil.Process().cpu_affinity()))
except:
    threads = os.cpu_count()

import logging
log = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,  # Set the minimum log level
    format="%(asctime)s - %(levelname)s - %(message)s",  # Include time, level, and message
    datefmt="%Y-%m-%d %H:%M:%S"  # Specify the date and time format
)

# Load the model and tokenizer
log.info("Loading model and tokenizer...")
model_name = "meta-llama/Llama-3.2-1B"
new_tokenizer_path = "/cmlscratch/astein0/efficient_tokenization_for_inference/tokenizers/Llama-3.2-tokenizer-genqa-math-empty-start-1000"

log.info("Loading base tokenizer...")
base_tokenizer = AutoTokenizer.from_pretrained(model_name)
log.info("Loading extended tokenizer...")
extended_tokenizer = AutoTokenizer.from_pretrained(new_tokenizer_path)

log.info("Loading dataset...")
ds_path = "/fs/cml-projects/llm-pretraining/datasets/raw/genqa/math"
ds = datasets.load_from_disk(ds_path)

log.info("Getting genqa data...")
ds = tokenize_simple.get_genqa_data(ds, track_role=True, batch_size=batch_size, threads=threads)
# log.info("Creating translation dataset...")
# ds = create_translation_dataset(ds, base_tokenizer, extended_tokenizer, batch_size, threads)

/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-03 16:53:15 - INFO - Loading model and tokenizer...
2025-03-03 16:53:15 - INFO - Loading base tokenizer...
2025-03-03 16:53:15 - INFO - Loading extended tokenizer...


tokenizer_object None
tokenizer_object None


2025-03-03 16:53:16 - INFO - Loading dataset...
2025-03-03 16:53:18 - INFO - Getting genqa data...
Traceback (most recent call last):
  File "/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/multiprocess/managers.py", line 599, in _run_server
    server.serve_forever()
  File "/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/multiprocess/managers.py", line 184, in serve_forever
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/sit

In [4]:
# Different ways to check the template
print("Direct template:", getattr(base_tokenizer, 'chat_template', None))
print("Config:", base_tokenizer.init_kwargs.get('chat_template'))

# Print all available attributes to find where the template might be stored
print("\nAll config attributes:")
for attr in dir(base_tokenizer):
    if not attr.startswith('_'):  # Skip private attributes
        try:
            value = getattr(base_tokenizer, attr)
            if 'template' in attr.lower():
                print(f"{attr}: {value}")
        except:
            pass

Direct template: None
Config: None

All config attributes:
apply_chat_template: <bound method PreTrainedTokenizerBase.apply_chat_template of PreTrainedTokenizerFast(name_or_path='meta-llama/Llama-3.2-1B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|

In [9]:
import tokenize_simple
import importlib
importlib.reload(tokenize_simple)
print(ds)

log.info("Creating translation dataset...")
# ds = tokenize_simple.create_translation_dataset(ds, base_tokenizer, extended_tokenizer, 2, threads)
ds = tokenize_simple.create_translation_dataset_with_template(ds, base_tokenizer, new_tokenizer_path, batch_size, threads)

2025-03-03 17:18:11 - INFO - Creating translation dataset...


Dataset({
    features: ['text'],
    num_rows: 515509
})
tokenizer_object None


/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/transformers/tokenization_utils

In [11]:
ds.select(range(40))["text"]

['<s>[INST] <<SYS>>\nYou are a helpful assistant.\n<</SYS>>\nRepeat the following text: <|user|> Find the area of a trapezoid with bases of 8 cm and 12 cm and an altitude of 6 cm. Both bases are extended by 3 cm to form a new trapezoid. Find the area of the new trapezoid. Express your answer in simplified form.<|assistant|> The area of the original trapezoid is:\n\n```\nArea = (8 cm + 12 cm) * 6 cm / 2 = 60 cm²\n```\n\nThe ratio of the bases of the new trapezoid to the old trapezoid is:\n\n```\nRatio = (8 cm + 3 cm + 12 cm + 3 cm) / (8 cm + 12 cm) = 1\n```\n\nSince the bases are in the same ratio, the areas of the trapezoids will also be in the same ratio, so the area of the new trapezoid is:\n\n```\nNew Area = 60 cm² * 1 = 60 cm²\n```<|user|> If the original trapezoid is partitioned into two congruent right triangles by the altitude, what is the area of each triangle?<|assistant|> The area of each triangle is:\n\n```\nTriangle Area = (8 cm + 12 cm) * 6 cm / 2 / 2 = 30 cm²\n``` [/INST]

In [1]:
import datasets
ds_path = "/cmlscratch/astein0/efficient_tokenization_for_inference/datasets/translation_tokenized"
ds = datasets.load_from_disk(ds_path)

/nfshomes/astein0/.pyenv/versions/3.10.4/envs/eff-tok/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels', 'loss_mask', 'num_tokens'],
    num_rows: 515509
})